## Deep Convolutional Generative Adverserial Network

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

### Image Preprocessing 

In [2]:
transform = transforms.Compose([
        transforms.Scale(36),
        transforms.RandomCrop(32),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5, 0.5, 0.5))])

### CIFAR-10 Dataset

In [4]:
train_dataset = dsets.CIFAR10(root='../data/',
                              train=True,
                              transform=transform,
                              download=True)

Files already downloaded and verified


### Data Loader (Input Pipeline)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

### 4x4 Convolution

In [6]:
def conv4x4(in_channels, out_channels, stride):
    return nn.Conv2d(in_channels, out_channels, kernel_size=4,
                     stride=stride, padding=1, bias=False)

### Discriminator Model

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            conv4x4(3, 16, 2),
            nn.LeakyReLU(0.2, inplace=True),
            conv4x4(16, 32, 2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),
            conv4x4(32, 64, 2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 1, kernel_size=4),
            nn.Sigmoid())
        
    def forward(self, x):
        out = self.model(x)
        out = out.view(out.size(0), -1)
        return out

### 4x4 Transpose Convolution

In [9]:
def conv_transpose4x4(in_channels, out_channels, stride=1, padding=1, bias=False):
    return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4,
                              stride=stride, padding=padding, bias=bias)

### Generator Model

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            conv_transpose4x4(128, 64, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            conv_transpose4x4(64, 32, 2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            conv_transpose4x4(32, 16, 2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            conv_transpose4x4(16, 3, 2, bias=True),
            nn.Tanh())
        
    def forward(self, x):
        x = x.view(x.size(0), 128, 1, 1)
        out = self.model(x)
        return out

In [11]:
discriminator = Discriminator()
generator = Generator()

### Loss and Optimizer 

In [12]:
criterion = nn.BCELoss()
lr = 0.0002
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

### Training 

In [17]:
for epoch in range(50):
    for i, (images, _) in enumerate(train_loader):
        images = Variable(images)
        real_labels = Variable(torch.ones(images.size(0)))
        fake_labels = Variable(torch.zeros(images.size(0)))
        
        # Train the discriminator
        discriminator.zero_grad()
        outputs = discriminator(images)
        real_loss = criterion(outputs, real_labels)
        real_score = outputs
        
        noise = Variable(torch.randn(images.size(0), 128))
        fake_images = generator(noise)
        outputs = discriminator(fake_images)
        fake_loss = criterion(outputs, fake_labels)
        fake_score = outputs
        
        d_loss = real_loss + fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        # Train the generator
        generator.zero_grad()
        noise = Variable(torch.randn(images.size(0), 128))
        fake_images = generator(noise)
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, '
                  'D(x): %.2f, D(G(Z)): %.2f'
                  %(epoch, 50, i+1, 500, d_loss.data[0], g_loss.data[0],
                    real_score.data.mean(), fake_score.data.mean()))
            
            # Save the sampled images
            torchvision.utils.save_image(fake_images.data,
                './data/fake_samples_%d_%d.png' %(epoch+1, i+1))

Epoch [0/50], Step[100/500], d_loss: 0.4927, g_loss: 2.5381, D(x): 0.81, D(G(Z)): 0.21
Epoch [0/50], Step[200/500], d_loss: 0.2641, g_loss: 2.5815, D(x): 0.91, D(G(Z)): 0.15
Epoch [0/50], Step[300/500], d_loss: 0.2285, g_loss: 2.8974, D(x): 0.89, D(G(Z)): 0.09
Epoch [0/50], Step[400/500], d_loss: 0.2652, g_loss: 2.7445, D(x): 0.88, D(G(Z)): 0.11
Epoch [0/50], Step[500/500], d_loss: 0.2633, g_loss: 2.6892, D(x): 0.91, D(G(Z)): 0.13
Epoch [1/50], Step[100/500], d_loss: 0.2262, g_loss: 3.7405, D(x): 0.90, D(G(Z)): 0.10
Epoch [1/50], Step[200/500], d_loss: 0.2972, g_loss: 4.0202, D(x): 0.85, D(G(Z)): 0.08
Epoch [1/50], Step[300/500], d_loss: 0.1545, g_loss: 4.0581, D(x): 0.93, D(G(Z)): 0.05
Epoch [1/50], Step[400/500], d_loss: 0.3759, g_loss: 3.3602, D(x): 0.82, D(G(Z)): 0.10
Epoch [1/50], Step[500/500], d_loss: 0.1195, g_loss: 4.4986, D(x): 0.95, D(G(Z)): 0.04
Epoch [2/50], Step[100/500], d_loss: 0.0842, g_loss: 4.9589, D(x): 0.97, D(G(Z)): 0.05
Epoch [2/50], Step[200/500], d_loss: 0.1652

### Save the Models

In [ ]:
torch.save(generator.state_dict(), './DCgenerator.pkl')
torch.save(discriminator.state_dict(), './DCdiscriminator.pkl')